# Sequence of dates

In [0]:
-- table to generate a sequence of dates
DROP TABLE IF EXISTS dev.bi.s12n_date_sequence_mgtv;
CREATE TABLE dev.bi.s12n_date_sequence_mgtv AS
(
  WITH start_end_dates AS
  (
    SELECT 
      base_phoenix_specialization_id,
      TRUNC(launch_date, 'MM') AS start_month,
      TRUNC(current_date, 'MM') AS end_month
  FROM
      dev.bi.s12n_estimated_total_b2c_revenue_mgtf 
  WHERE 
    launch_date IS NOT NULL -- removes S12Ns that haven't been launched yet.
  GROUP BY 1, 2, 3
  )

  SELECT 
    base_phoenix_specialization_id, 
    start_month,
    end_month,
    EXPLODE(SEQUENCE(start_month, end_month, INTERVAL 1 MONTH)) AS date_month
  FROM start_end_dates
)


num_affected_rows,num_inserted_rows


# Revenue Aggregation table

In [0]:
DROP TABLE IF EXISTS dev.bi.s12n_estimated_b2c_revenue_aggregated_mgtv;
CREATE TABLE dev.bi.s12n_estimated_b2c_revenue_aggregated_mgtv AS
(
WITH b2c_revenue AS
(
  SELECT 
      rev_rec_month_start_dt,
      TRUNC(launch_date, 'MM') AS launch_month,
      base_phoenix_specialization_id,
      sum(b2c_revenue) as b2c_revenue
  FROM 
      dev.bi.s12n_estimated_total_b2c_revenue_mgtf
    GROUP BY 1,2,3
),

-- Join this with the date sequence table
filled_b2c_revenue_data AS
(
SELECT 
    COALESCE(br.rev_rec_month_start_dt, ds.date_month) AS date_month,
    COALESCE(br.launch_month, ds.start_month) AS launch_month,
    COALESCE(br.base_phoenix_specialization_id, ds.base_phoenix_specialization_id) AS base_phoenix_specialization_id,
    COALESCE(br.b2c_revenue, 0) AS b2c_revenue
FROM 
    b2c_revenue br
RIGHT JOIN
    dev.bi.s12n_date_sequence_mgtv ds
ON 
    br.base_phoenix_specialization_id = ds.base_phoenix_specialization_id
AND
    br.rev_rec_month_start_dt = ds.date_month
)

-- Add recognized revenue aggregate from the current month to the next 11 months (12 months total)
-- The n12m value when date_month = launch_month is the Y1 recognized revenue
SELECT 
    date_month,
    launch_month,
    base_phoenix_specialization_id,
    b2c_revenue,
    SUM(b2c_revenue) OVER (PARTITION BY base_phoenix_specialization_id ORDER BY date_month
                               ROWS BETWEEN CURRENT ROW AND 11 FOLLOWING) AS rolling_n12m_b2c_revenue
FROM 
filled_b2c_revenue_data
where date_month < to_date(date_trunc('month',current_date))
)

num_affected_rows,num_inserted_rows
